In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
#import plotly
import tensorflow as tf
import matplotlib.pyplot as plt
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv("../input/sunspots/Sunspots.csv")
ds=df[['Unnamed: 0','Monthly Mean Total Sunspot Number']]

In [ ]:
df.head()

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(y=df["Monthly Mean Total Sunspot Number"], x=df["Date"],
                    mode='lines',
                    name='lines',line_color="red"))
fig.show()

In [ ]:
series=df["Monthly Mean Total Sunspot Number"].to_numpy()
s=series
timestamp=df['Unnamed: 0'].to_numpy()
split=2600
train=series[:split]
train_time=timestamp[:split]
test=series[split:]
test_time=timestamp[split:]

In [ ]:
series.shape

In [ ]:
def window(series,window_size,batch_size,shuffle):
    series=tf.expand_dims(series,axis=1)
    dataset=tf.data.Dataset.from_tensor_slices(series)
    dataset=dataset.window(window_size+1,shift=1,drop_remainder=True)
    dataset=dataset.flat_map(lambda window:window.batch(window_size+1))
    dataset=dataset.map((lambda window:(window[:-1],window[-1])))
    dataset=dataset.shuffle(shuffle)
    dataset=dataset.batch(batch_size).prefetch(1)
    return dataset

In [ ]:
tf.keras.backend.clear_session()
dataset=window(train,60,250,2600)


In [ ]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(60, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(60,return_sequences=True)),
  tf.keras.layers.Dense(30,activation='relu'),
  tf.keras.layers.Dense(10,activation='relu'),
                                  
                                  
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 400.0)
])
model.compile(loss=tf.keras.losses.Huber(),optimizer="adam",metrics=["mae"])
model.fit(dataset,epochs=100)

In [ ]:
print(model.summary())
window_size=60

In [ ]:
forecast = []
results = []
s=tf.expand_dims(s,axis=1)
for time in range(len(series) - window_size):
    
  forecast.append(model.predict(s[time:time + window_size][np.newaxis]))
  

forecast = forecast[split-window_size:]
results = np.array(forecast)[:, 0, 0]



In [ ]:

plt.figure(figsize=(10, 6))

plt.plot(test_time, test)
plt.plot(test_time, results)

